In [1]:
from plaid_bridges.dataloaders import PlaidDataLoader, HeterogeneousCollater, HomogeneousCollater

from datasets import load_dataset, load_from_disk
from plaid.containers.sample import Sample
import pickle
from plaid.bridges.huggingface_bridge import (
    huggingface_dataset_to_plaid,
    huggingface_description_to_problem_definition,
)

c:\Users\d582428\AppData\Local\miniforge3\envs\plaid-bridges\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[2025-08-19 13:16:11,706:INFO:config.py:<module>(54)]:PyTorch version 2.8.0 available.


## Case with homogeneous samples

In [ ]:
hf_dataset = load_dataset("PLAID-datasets/VKI-LS59", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train")[:10]

dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

Converting huggingface dataset to plaid dataset...


100%|██████████| 10/10 [00:07<00:00,  1.26it/s]


In [3]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

in_feature_identifiers = [scalar_features[0], field_features[0]]
out_feature_identifiers = [field_features[1], scalar_features[1]]

print(in_feature_identifiers)
print(out_feature_identifiers)

Dataset(10 samples, 8 scalars, 0 time_series, 8 fields)
[{'type': 'scalar', 'name': np.str_('Pr')}, {'type': 'field', 'name': 'nut', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}]
[{'type': 'field', 'name': 'roe', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'scalar', 'name': np.str_('Q')}]


In [4]:
loader = PlaidDataLoader(dataset, in_feature_identifiers = in_feature_identifiers, out_feature_identifiers = out_feature_identifiers, collate_fn=HomogeneousCollater, batch_size = 2, shuffle = True, )

def _make_hashable(feat_dict: dict):
    return tuple(sorted(feat_dict.items()))

batch = next(iter(loader))
print("scalars =", batch[0][_make_hashable(in_feature_identifiers[0])])
print("fields =", batch[1][_make_hashable(out_feature_identifiers[0])], " | >>>> tensor:", batch[1][_make_hashable(out_feature_identifiers[0])].shape)

scalars = tensor([0.9636, 0.9616], dtype=torch.float64)
fields = tensor([[2.6890, 2.6890, 2.6890,  ..., 1.9656, 1.9655, 1.9654],
        [3.3818, 3.3818, 3.3817,  ..., 2.7646, 2.7646, 2.7645]],
       dtype=torch.float64)  | >>>> tensor: torch.Size([2, 36421])


## Case with heterogeneous samples

In [ ]:
hf_dataset = load_dataset("PLAID-datasets/tensile2d", split="all_samples[:10]")
sample = Sample.model_validate(pickle.loads(hf_dataset[0]["sample"]))

pb_def = huggingface_description_to_problem_definition(hf_dataset.info.description)
ids = pb_def.get_split("train_500")[:10]


dataset, _ = huggingface_dataset_to_plaid(hf_dataset, ids=ids, processes_number=5)

Converting huggingface dataset to plaid dataset...


100%|██████████| 10/10 [00:06<00:00,  1.64it/s]


In [6]:
print(dataset)
all_feat_ids = dataset[0].get_all_features_identifiers()

scalar_features = [f for f in all_feat_ids if "scalar" in f.values()]
field_features = [f for f in all_feat_ids if "field" in f.values()]

in_feature_identifiers = [scalar_features[0], field_features[0]]
out_feature_identifiers = [field_features[1], scalar_features[1]]

print(in_feature_identifiers)
print(out_feature_identifiers)

Dataset(10 samples, 10 scalars, 0 time_series, 6 fields)
[{'type': 'scalar', 'name': np.str_('P')}, {'type': 'field', 'name': 'sig22', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}]
[{'type': 'field', 'name': 'q', 'base_name': 'Base_2_2', 'zone_name': 'Zone', 'location': 'Vertex', 'time': np.float64(0.0)}, {'type': 'scalar', 'name': np.str_('max_U2_top')}]


In [9]:
loader = PlaidDataLoader(dataset, in_feature_identifiers = in_feature_identifiers, out_feature_identifiers = out_feature_identifiers, collate_fn=HeterogeneousCollater, batch_size = 2, shuffle = True)

def _make_hashable(feat_dict: dict):
    return tuple(sorted(feat_dict.items()))

batch = next(iter(loader))
print("scalars =", batch[0][_make_hashable(in_feature_identifiers[0])])
print("fields =", batch[1][_make_hashable(out_feature_identifiers[0])], " | >>>> list:", type(batch[1][_make_hashable(out_feature_identifiers[0])]))

scalars = [np.float64(-47.79), np.float64(-48.03)]
fields = [array([0.00000000e+00, 0.00000000e+00, 0.00000000e+00, ...,
       7.33460474e-05, 1.30227287e-04, 0.00000000e+00], shape=(6184,)), array([0., 0., 0., ..., 0., 0., 0.], shape=(6161,))]  | >>>> list: <class 'list'>
